In [563]:
import os
import re
import string
import nltk
import datetime
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [7]:
data_directory = os.getcwd() + '/final_data'

cwd, files, filenames = next(os.walk(data_directory))
filenames = [file for file in filenames if file[-4:] == '.csv']

print(cwd)
print(filenames)

/Users/eddychiao/Documents/Georgia Tech/CS/GaTech-CS/MSCS/CS 6474/project/final_data
['x_train.csv', 'y_train.csv', 'y_test.csv', 'x_test.csv']


In [30]:
x_train = pd.read_csv(data_directory + '/x_train.csv')
y_train = pd.read_csv(data_directory + '/y_train.csv')
x_test = pd.read_csv(data_directory + '/x_test.csv')
y_test = pd.read_csv(data_directory + '/y_test.csv')

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

list_x_features = list(x_train.head(0))
list_y_features = list(y_train.head(0))

print(list_x_features)
print()
print(list_y_features)

(938, 165)
(938, 18)
(105, 165)
(105, 18)
['Unnamed: 0', 'timestamp', 'Sentiment_Sum', 'Sentiment_Mean', 'Sentiment_Min', 'Sentiment_Max', 'Sentiment_Median', 'Total_Number_Negative', 'Total_Number_Neutral', 'Total_Number_Positive', 'Fraction_Negative', 'Fraction_Positive', 'Fraction_Neutral', 'Mean_Sentiment_Last_3_Time_Period', 'Mean_Fraction_Negative_Last_3_Time_Period', 'Mean_Fraction_Positive_Last_3_Time_Period', 'Mean_Fraction_Neutral_Last_3_Time_Period', 'Mean_Sentiment_Sum_Last_3_Time_Period', 'Total_Sentiment_Sum_Last_3_Time_Period', 'Max_Sentiment_Last_3_Time_Period', 'Min_Sentiment_Last_3_Time_Period', 'Total_BUY_Occurences', 'Mean_BUY_Occurences', 'Median_BUY_Occurences', 'Max_BUY_Occurences', 'Min_BUY_Occurences', 'Mean_Fraction_of_post_BUY', 'Median_Fraction_of_post_BUY', 'Max_Fraction_of_post_BUY', 'Min_Fraction_of_post_BUY', 'Total_SELL_Occurences', 'Mean_SELL_Occurences', 'Median_SELL_Occurences', 'Max_SELL_Occurences', 'Min_SELL_Occurences', 'Mean_Fraction_of_post_SEL

In [25]:
sentiment_features = list_x_features[2:21]
stopword_features = list_x_features[21:57]
ngram_features = list_x_features[57:]

# Model 1: Sentiment

In [43]:
sentiment_train = x_train.copy()[sentiment_features].fillna(method='backfill')
sentiment_test = x_test.copy()[sentiment_features].fillna(method='backfill')
sentiment_train

,Sentiment_Sum,Sentiment_Mean,Sentiment_Min,Sentiment_Max,Sentiment_Median,Total_Number_Negative,Total_Number_Neutral,Total_Number_Positive,Fraction_Negative,Fraction_Positive,Fraction_Neutral,Mean_Sentiment_Last_3_Time_Period,Mean_Fraction_Negative_Last_3_Time_Period,Mean_Fraction_Positive_Last_3_Time_Period,Mean_Fraction_Neutral_Last_3_Time_Period,Mean_Sentiment_Sum_Last_3_Time_Period,Total_Sentiment_Sum_Last_3_Time_Period,Max_Sentiment_Last_3_Time_Period,Min_Sentiment_Last_3_Time_Period
0,-7.0908,-0.050289,-0.8655,0.6124,0.0,40,74,27,0.283688,0.191489,0.524823,-0.015660,0.257534,0.258297,0.484169,-1.775100,-5.3253,0.8789,-0.8885
1,-2.0048,-0.026379,-0.8720,0.7717,0.0,21,34,21,0.276316,0.276316,0.447368,-0.015660,0.257534,0.258297,0.484169,-1.775100,-5.3253,0.8789,-0.8885
2,3.7703,0.029687,-0.8885,0.8789,0.0,27,61,39,0.212598,0.307087,0.480315,-0.015660,0.257534,0.258297,0.484169,-1.775100,-5.3253,0.8789,-0.8885
3,-7.3375,-0.033813,-0.8271,0.8689,0.0,58,115,44,0.267281,0.202765,0.529954,-0.010168,0.252065,0.262056,0.485879,-1.857333,-5.5720,0.8789,-0.8885
4,0.3388,0.003986,-0.7430,0.7351,0.0,18,44,23,0.211765,0.270588,0.517647,-0.000047,0.230548,0.260147,0.509305,-1.076133,-3.2284,0.8789,-0.8885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,12.7906,0.063953,-0.7845,0.8961,0.0,35,113,52,0.175000,0.260000,0.565000,0.084570,0.176998,0.327036,0.495966,22.355267,67.0658,0.9100,-0.9270
934,31.3224,0.086526,-0.8074,0.8360,0.0,51,197,114,0.140884,0.314917,0.544199,0.076040,0.163809,0.306695,0.529496,27.825600,83.4768,0.9100,-0.9270
935,24.1370,0.092126,-0.8834,0.8655,0.0,46,122,94,0.175573,0.358779,0.465649,0.080868,0.163819,0.311232,0.524949,22.750000,68.2500,0.8961,-0.8834
936,3.2163,0.071473,-0.8225,0.8625,0.0,10,18,17,0.222222,0.377778,0.400000,0.083375,0.179560,0.350491,0.469949,19.558567,58.6757,0.8655,-0.8834


In [39]:
y_train

,Unnamed: 0,SPYClose,SPYVolume,DJIAClose,DJIAVolume,AAPLClose,AAPLVolume,GOOGClose,GOOGVolume,AMCClose,AMCVolume,GMEClose,GMEVolume,BBClose,BBVolume,NOKClose,NOKVolume,timestamp
0,0,0,0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,2020-04-01 09:30:00
1,1,0,0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,2020-04-01 12:30:00
2,2,1,1,1.0,1.0,1,1,1,1,0,1,0,0,0,0,1,1,2020-04-02 00:00:00
3,3,0,1,1.0,1.0,0,1,0,1,1,1,1,0,1,1,1,1,2020-04-02 09:30:00
4,4,0,0,0.0,0.0,0,0,1,0,0,0,0,1,0,0,0,0,2020-04-02 12:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,1038,1,0,0.0,0.0,1,0,1,0,0,0,0,0,0,0,1,0,2021-03-29 12:30:00
934,1039,0,1,0.0,0.0,0,1,1,1,0,0,1,1,0,1,0,1,2021-03-30 00:00:00
935,1040,1,1,0.0,0.0,1,1,1,1,0,1,0,0,0,0,0,0,2021-03-31 00:00:00
936,1041,0,0,0.0,0.0,1,1,1,0,0,1,1,1,1,1,1,1,2021-03-31 09:30:00


In [85]:
AAPL_volume_train = y_train.copy()['AAPLVolume']
AAPL_close_train = y_train.copy()['AAPLClose']
AAPL_volume_test = y_test.copy()['AAPLVolume']
AAPL_close_test = y_test.copy()['AAPLClose']

GOOG_volume_train = y_train.copy()['GOOGVolume']
GOOG_close_train = y_train.copy()['GOOGClose']
GOOG_volume_test = y_test.copy()['GOOGVolume']
GOOG_close_test = y_test.copy()['GOOGClose']

NOK_volume_train = y_train.copy()['NOKVolume']
NOK_close_train = y_train.copy()['NOKClose']
NOK_volume_test = y_test.copy()['NOKVolume']
NOK_close_test = y_test.copy()['NOKClose']

## Volume Training APPL LinReg

In [200]:
lin_model = LinearRegression()
lin_model.fit(sentiment_train, AAPL_volume_train)
volume_pred = lin_model.predict(sentiment_test)

# The coefficients
print('Coefficients: \n', lin_model.coef_)

# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(AAPL_volume_test, volume_pred))

rounded_volume_pred = volume_pred.copy()
rounded_volume_pred[rounded_volume_pred > 0.5] = 1
rounded_volume_pred[rounded_volume_pred <= 0.5] = 0

# Accuacy score
print('Accuracy:', accuracy_score(AAPL_volume_test, rounded_volume_pred))
print('Precision:', precision_score(AAPL_volume_test, rounded_volume_pred))
print('Recall:', recall_score(AAPL_volume_test, rounded_volume_pred))
print('F1:', f1_score(AAPL_volume_test, rounded_volume_pred))
print('AUC:', roc_auc_score(AAPL_volume_test, rounded_volume_pred))
print()
print('Confusion Matrix:\n', confusion_matrix(AAPL_volume_test, rounded_volume_pred))
print('Report:\n', classification_report(AAPL_volume_test, rounded_volume_pred))

Coefficients: 
 [ 8.49970167e-03 -1.39615095e+00 -4.64523309e-01  8.86330195e-01
 -5.98159905e-01  2.52209855e-03  3.70898881e-04 -3.14879297e-03
  1.90487993e+06  1.90488116e+06  1.90488010e+06  9.15585627e-01
 -9.20708654e+05 -9.20709491e+05 -9.20708379e+05 -1.21624337e+05
  4.05414443e+04 -6.63931031e-01  8.52390219e-01]
Mean squared error: 0.26
Accuracy: 0.5904761904761905
Precision: 0.6111111111111112
Recall: 0.7457627118644068
F1: 0.6717557251908397
AUC: 0.5685335298452469

Confusion Matrix:
 [[18 28]
 [15 44]]
Report:
               precision    recall  f1-score   support

           0       0.55      0.39      0.46        46
           1       0.61      0.75      0.67        59

    accuracy                           0.59       105
   macro avg       0.58      0.57      0.56       105
weighted avg       0.58      0.59      0.58       105



## Volume Training APPL RandForest

In [199]:
rand_forest_model = RandomForestClassifier(max_depth=100, min_samples_leaf=15, n_estimators=100)
rand_forest_model.fit(sentiment_train, AAPL_volume_train)
volume_pred = rand_forest_model.predict(sentiment_test)

# Accuacy score
print('Accuracy:', accuracy_score(AAPL_volume_test, volume_pred))
print('Precision:', precision_score(AAPL_volume_test, volume_pred))
print('Recall:', recall_score(AAPL_volume_test, volume_pred))
print('F1:', f1_score(AAPL_volume_test, volume_pred))
print('AUC:', roc_auc_score(AAPL_volume_test, volume_pred))
print()
print('Confusion Matrix:\n', confusion_matrix(AAPL_volume_test, volume_pred))
print('Report:\n', classification_report(AAPL_volume_test, volume_pred))

Accuracy: 0.6857142857142857
Precision: 0.7096774193548387
Recall: 0.7457627118644068
F1: 0.7272727272727273
AUC: 0.6772291820191599

Confusion Matrix:
 [[28 18]
 [15 44]]
Report:
               precision    recall  f1-score   support

           0       0.65      0.61      0.63        46
           1       0.71      0.75      0.73        59

    accuracy                           0.69       105
   macro avg       0.68      0.68      0.68       105
weighted avg       0.68      0.69      0.68       105



## Close Training APPL LinReg

In [201]:
lin_model = LinearRegression()
lin_model.fit(sentiment_train, AAPL_close_train)
close_pred = lin_model.predict(sentiment_test)

# The coefficients
print('Coefficients: \n', lin_model.coef_)

# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(AAPL_close_test, close_pred))

rounded_close_pred = close_pred.copy()
rounded_close_pred[rounded_close_pred > 0.5] = 1
rounded_close_pred[rounded_close_pred <= 0.5] = 0

# Accuacy score
print('Accuracy:', accuracy_score(AAPL_close_test, rounded_close_pred))
print('Precision:', precision_score(AAPL_close_test, rounded_close_pred))
print('Recall:', recall_score(AAPL_close_test, rounded_close_pred))
print('F1:', f1_score(AAPL_close_test, rounded_close_pred))
print('AUC:', roc_auc_score(AAPL_close_test, rounded_close_pred))
print()
print('Confusion Matrix:\n', confusion_matrix(AAPL_close_test, rounded_close_pred))
print('Report:\n', classification_report(AAPL_close_test, rounded_close_pred))

Coefficients: 
 [-4.11214993e-03 -1.38051622e+00  1.09693170e-01  1.64719675e-01
 -6.62062254e-01 -1.94895991e-03 -4.09852311e-04  2.71223262e-03
  1.35563051e+06  1.35563171e+06  1.35563142e+06  9.32671861e-01
 -2.50726940e+06 -2.50727057e+06 -2.50727113e+06  1.17459047e+05
 -3.91530155e+04 -5.81921658e-01  2.89383315e-01]
Mean squared error: 0.24
Accuracy: 0.5619047619047619
Precision: 0.5757575757575758
Recall: 0.9344262295081968
F1: 0.7125000000000001
AUC: 0.48994038748137103

Confusion Matrix:
 [[ 2 42]
 [ 4 57]]
Report:
               precision    recall  f1-score   support

           0       0.33      0.05      0.08        44
           1       0.58      0.93      0.71        61

    accuracy                           0.56       105
   macro avg       0.45      0.49      0.40       105
weighted avg       0.47      0.56      0.45       105



## Close Training APPL RandForest

In [204]:
rand_forest_model = RandomForestClassifier(max_depth=100, min_samples_leaf=15, n_estimators=100)
rand_forest_model.fit(sentiment_train, AAPL_close_train)
close_pred = rand_forest_model.predict(sentiment_test)

# Accuacy score
print('Accuracy:', accuracy_score(AAPL_close_test, close_pred))
print('Precision:', precision_score(AAPL_close_test, close_pred))
print('Recall:', recall_score(AAPL_close_test, close_pred))
print('F1:', f1_score(AAPL_close_test, close_pred))
print('AUC:', roc_auc_score(AAPL_close_test, close_pred))
print()
print('Confusion Matrix:\n', confusion_matrix(AAPL_close_test, close_pred))
print('Report:\n', classification_report(AAPL_close_test, close_pred))

Accuracy: 0.580952380952381
Precision: 0.5934065934065934
Recall: 0.8852459016393442
F1: 0.7105263157894737
AUC: 0.5221684053651267

Confusion Matrix:
 [[ 7 37]
 [ 7 54]]
Report:
               precision    recall  f1-score   support

           0       0.50      0.16      0.24        44
           1       0.59      0.89      0.71        61

    accuracy                           0.58       105
   macro avg       0.55      0.52      0.48       105
weighted avg       0.55      0.58      0.51       105



--------------------

## Volume Training GOOG LinReg

In [205]:
lin_model = LinearRegression()
lin_model.fit(sentiment_train, GOOG_volume_train)
volume_pred = lin_model.predict(sentiment_test)

# The coefficients
print('Coefficients: \n', lin_model.coef_)

# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(GOOG_volume_test, volume_pred))

rounded_volume_pred = volume_pred.copy()
rounded_volume_pred[rounded_volume_pred > 0.5] = 1
rounded_volume_pred[rounded_volume_pred <= 0.5] = 0

# Accuacy score
print('Accuracy:', accuracy_score(GOOG_volume_test, rounded_volume_pred))
print('Precision:', precision_score(GOOG_volume_test, rounded_volume_pred))
print('Recall:', recall_score(GOOG_volume_test, rounded_volume_pred))
print('F1:', f1_score(GOOG_volume_test, rounded_volume_pred))
print('AUC:', roc_auc_score(GOOG_volume_test, rounded_volume_pred))
print()
print('Confusion Matrix:\n', confusion_matrix(GOOG_volume_test, rounded_volume_pred))
print('Report:\n', classification_report(GOOG_volume_test, rounded_volume_pred))

Coefficients: 
 [ 1.96733527e-03  4.61415185e-01 -5.48270414e-01  4.68340857e-01
 -3.62750163e-01 -3.50322674e-04  1.47119307e-04  1.86971669e-04
  5.22559967e+06  5.22560007e+06  5.22559977e+06  4.33175670e-01
  5.30683328e+06  5.30683267e+06  5.30683228e+06 -1.15258605e+05
  3.84195335e+04 -3.58721611e-01  6.19817554e-01]
Mean squared error: 0.23
Accuracy: 0.638095238095238
Precision: 0.6444444444444445
Recall: 0.90625
F1: 0.7532467532467533
AUC: 0.5628810975609756

Confusion Matrix:
 [[ 9 32]
 [ 6 58]]
Report:
               precision    recall  f1-score   support

           0       0.60      0.22      0.32        41
           1       0.64      0.91      0.75        64

    accuracy                           0.64       105
   macro avg       0.62      0.56      0.54       105
weighted avg       0.63      0.64      0.58       105



## Volume Training GOOG RandForest

In [206]:
rand_forest_model = RandomForestClassifier(max_depth=100, min_samples_leaf=15, n_estimators=100)
rand_forest_model.fit(sentiment_train, GOOG_volume_train)
volume_pred = rand_forest_model.predict(sentiment_test)

# Accuacy score
print('Accuracy:', accuracy_score(GOOG_volume_test, volume_pred))
print('Precision:', precision_score(GOOG_volume_test, volume_pred))
print('Recall:', recall_score(GOOG_volume_test, volume_pred))
print('F1:', f1_score(AAPL_volume_test, volume_pred))
print('AUC:', roc_auc_score(AAPL_volume_test, volume_pred))
print()
print('Confusion Matrix:\n', confusion_matrix(GOOG_volume_test, volume_pred))
print('Report:\n', classification_report(GOOG_volume_test, volume_pred))

Accuracy: 0.7238095238095238
Precision: 0.7108433734939759
Recall: 0.921875
F1: 0.7746478873239436
AUC: 0.6617538688282977

Confusion Matrix:
 [[17 24]
 [ 5 59]]
Report:
               precision    recall  f1-score   support

           0       0.77      0.41      0.54        41
           1       0.71      0.92      0.80        64

    accuracy                           0.72       105
   macro avg       0.74      0.67      0.67       105
weighted avg       0.74      0.72      0.70       105



## Close Training GOOG LinReg

In [209]:
lin_model = LinearRegression()
lin_model.fit(sentiment_train, GOOG_close_train)
close_pred = lin_model.predict(sentiment_test)

# The coefficients
print('Coefficients: \n', lin_model.coef_)

# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(GOOG_close_test, close_pred))

rounded_close_pred = close_pred.copy()
rounded_close_pred[rounded_close_pred > 0.5] = 1
rounded_close_pred[rounded_close_pred <= 0.5] = 0

# Accuacy score
print('Accuracy:', accuracy_score(GOOG_close_test, rounded_close_pred))
print('Precision:', precision_score(GOOG_close_test, rounded_close_pred))
print('Recall:', recall_score(GOOG_close_test, rounded_close_pred))
print('F1:', f1_score(GOOG_close_test, rounded_close_pred))
print('AUC:', roc_auc_score(GOOG_close_test, rounded_close_pred))
print()
print('Confusion Matrix:\n', confusion_matrix(GOOG_close_test, rounded_close_pred))
print('Report:\n', classification_report(GOOG_close_test, rounded_close_pred))

Coefficients: 
 [ 4.25612472e-03 -5.73735048e-02 -9.50597601e-02 -5.00316764e-02
 -9.31707402e-02  2.32477962e-03  3.79592723e-04 -3.06879146e-03
  3.38581870e+06  3.38581904e+06  3.38581877e+06 -4.27847934e-01
  2.99502203e+06  2.99502139e+06  2.99502179e+06 -8.36920438e+04
  2.78973485e+04 -1.93192849e-01  4.33599371e-01]
Mean squared error: 0.25
Accuracy: 0.5333333333333333
Precision: 0.5
Recall: 0.9183673469387755
F1: 0.6474820143884893
AUC: 0.5573979591836734

Confusion Matrix:
 [[11 45]
 [ 4 45]]
Report:
               precision    recall  f1-score   support

           0       0.73      0.20      0.31        56
           1       0.50      0.92      0.65        49

    accuracy                           0.53       105
   macro avg       0.62      0.56      0.48       105
weighted avg       0.62      0.53      0.47       105



## Close Training GOOG RandForest

In [211]:
rand_forest_model = RandomForestClassifier(max_depth=100, min_samples_leaf=15, n_estimators=100)
rand_forest_model.fit(sentiment_train, GOOG_close_train)
close_pred = rand_forest_model.predict(sentiment_test)

# Accuacy score
print('Accuracy:', accuracy_score(GOOG_close_test, close_pred))
print('Precision:', precision_score(GOOG_close_test, close_pred))
print('Recall:', recall_score(GOOG_close_test, close_pred))
print('F1:', f1_score(GOOG_close_test, close_pred))
print('AUC:', roc_auc_score(GOOG_close_test, close_pred))
print()
print('Confusion Matrix:\n', confusion_matrix(GOOG_close_test, close_pred))
print('Report:\n', classification_report(GOOG_close_test, close_pred))

Accuracy: 0.5619047619047619
Precision: 0.5189873417721519
Recall: 0.8367346938775511
F1: 0.640625
AUC: 0.5790816326530612

Confusion Matrix:
 [[18 38]
 [ 8 41]]
Report:
               precision    recall  f1-score   support

           0       0.69      0.32      0.44        56
           1       0.52      0.84      0.64        49

    accuracy                           0.56       105
   macro avg       0.61      0.58      0.54       105
weighted avg       0.61      0.56      0.53       105



------

## Volume Training NOK LinReg

In [212]:
lin_model = LinearRegression()
lin_model.fit(sentiment_train, NOK_volume_train)
volume_pred = lin_model.predict(sentiment_test)

# The coefficients
print('Coefficients: \n', lin_model.coef_)

# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(NOK_volume_test, volume_pred))

rounded_volume_pred = volume_pred.copy()
rounded_volume_pred[rounded_volume_pred > 0.5] = 1
rounded_volume_pred[rounded_volume_pred <= 0.5] = 0

# Accuacy score
print('Accuracy:', accuracy_score(NOK_volume_test, rounded_volume_pred))
print('Precision:', precision_score(NOK_volume_test, rounded_volume_pred))
print('Recall:', recall_score(NOK_volume_test, rounded_volume_pred))
print('F1:', f1_score(NOK_volume_test, rounded_volume_pred))
print('AUC:', roc_auc_score(NOK_volume_test, rounded_volume_pred))
print()
print('Confusion Matrix:\n', confusion_matrix(NOK_volume_test, rounded_volume_pred))
print('Report:\n', classification_report(NOK_volume_test, rounded_volume_pred))

Coefficients: 
 [-1.65395699e-03 -5.59645362e-01 -3.17115978e-01  7.34647215e-01
 -2.02465035e-01 -2.22385796e-03 -3.91323458e-04  3.01138871e-03
  6.23049850e+06  6.23049902e+06  6.23049848e+06 -9.70700288e-01
  4.68842176e+05  4.68842261e+05  4.68842330e+05  3.13797744e+04
 -1.04599264e+04 -1.20645526e+00  6.46629850e-01]
Mean squared error: 0.24
Accuracy: 0.6666666666666666
Precision: 0.6933333333333334
Recall: 0.8125
F1: 0.7482014388489209
AUC: 0.6257621951219512

Confusion Matrix:
 [[18 23]
 [12 52]]
Report:
               precision    recall  f1-score   support

           0       0.60      0.44      0.51        41
           1       0.69      0.81      0.75        64

    accuracy                           0.67       105
   macro avg       0.65      0.63      0.63       105
weighted avg       0.66      0.67      0.65       105



## Volume Training NOK RandForest

In [214]:
rand_forest_model = RandomForestClassifier(max_depth=100, min_samples_leaf=15, n_estimators=100)
rand_forest_model.fit(sentiment_train, NOK_volume_train)
volume_pred = rand_forest_model.predict(sentiment_test)

# Accuacy score
print('Accuracy:', accuracy_score(NOK_volume_test, volume_pred))
print('Precision:', precision_score(NOK_volume_test, volume_pred))
print('Recall:', recall_score(NOK_volume_test, volume_pred))
print('F1:', f1_score(NOK_volume_test, volume_pred))
print('AUC:', roc_auc_score(NOK_volume_test, volume_pred))
print()
print('Confusion Matrix:\n', confusion_matrix(NOK_volume_test, volume_pred))
print('Report:\n', classification_report(NOK_volume_test, volume_pred))

Accuracy: 0.7523809523809524
Precision: 0.8064516129032258
Recall: 0.78125
F1: 0.7936507936507936
AUC: 0.7442835365853657

Confusion Matrix:
 [[29 12]
 [14 50]]
Report:
               precision    recall  f1-score   support

           0       0.67      0.71      0.69        41
           1       0.81      0.78      0.79        64

    accuracy                           0.75       105
   macro avg       0.74      0.74      0.74       105
weighted avg       0.75      0.75      0.75       105



## Close Training NOK LinReg

In [215]:
lin_model = LinearRegression()
lin_model.fit(sentiment_train, NOK_close_train)
close_pred = lin_model.predict(sentiment_test)

# The coefficients
print('Coefficients: \n', lin_model.coef_)

# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(NOK_close_test, close_pred))

rounded_close_pred = close_pred.copy()
rounded_close_pred[rounded_close_pred > 0.5] = 1
rounded_close_pred[rounded_close_pred <= 0.5] = 0

# Accuacy score
print('Accuracy:', accuracy_score(NOK_close_test, rounded_close_pred))
print('Precision:', precision_score(NOK_close_test, rounded_close_pred))
print('Recall:', recall_score(NOK_close_test, rounded_close_pred))
print('F1:', f1_score(NOK_close_test, rounded_close_pred))
print('AUC:', roc_auc_score(NOK_close_test, rounded_close_pred))
print()
print('Confusion Matrix:\n', confusion_matrix(NOK_close_test, rounded_close_pred))
print('Report:\n', classification_report(NOK_close_test, rounded_close_pred))

Coefficients: 
 [-5.67235852e-03 -2.77744910e-01 -8.77422214e-02  6.49235979e-02
  1.71945447e-01 -2.34607173e-03 -1.06999695e-03  4.36930446e-03
  7.19366534e+06  7.19366551e+06  7.19366543e+06 -1.29116534e+00
 -1.69715342e+06 -1.69715268e+06 -1.69715322e+06  1.87677909e+05
 -6.25593028e+04 -7.29318650e-01  5.24055436e-03]
Mean squared error: 0.26
Accuracy: 0.5333333333333333
Precision: 0.5529411764705883
Recall: 0.8103448275862069
F1: 0.6573426573426574
AUC: 0.5009170946441672

Confusion Matrix:
 [[ 9 38]
 [11 47]]
Report:
               precision    recall  f1-score   support

           0       0.45      0.19      0.27        47
           1       0.55      0.81      0.66        58

    accuracy                           0.53       105
   macro avg       0.50      0.50      0.46       105
weighted avg       0.51      0.53      0.48       105



## Close Training NOK RandForest

In [220]:
rand_forest_model = RandomForestClassifier(max_depth=100, min_samples_leaf=15, n_estimators=100)
rand_forest_model.fit(sentiment_train, NOK_close_train)
close_pred = rand_forest_model.predict(sentiment_test)

# Accuacy score
print('Accuracy:', accuracy_score(NOK_close_test, close_pred))
print('Precision:', precision_score(NOK_close_test, close_pred))
print('Recall:', recall_score(NOK_close_test, close_pred))
print('F1:', f1_score(NOK_close_test, close_pred))
print('AUC:', roc_auc_score(NOK_close_test, close_pred))
print()
print('Confusion Matrix:\n', confusion_matrix(NOK_close_test, close_pred))
print('Report:\n', classification_report(NOK_close_test, close_pred))

Accuracy: 0.5714285714285714
Precision: 0.5915492957746479
Recall: 0.7241379310344828
F1: 0.6511627906976745
AUC: 0.5535583272193689

Confusion Matrix:
 [[18 29]
 [16 42]]
Report:
               precision    recall  f1-score   support

           0       0.53      0.38      0.44        47
           1       0.59      0.72      0.65        58

    accuracy                           0.57       105
   macro avg       0.56      0.55      0.55       105
weighted avg       0.56      0.57      0.56       105



---

# Model 2: Stopwords

In [221]:
stopword_train = x_train.copy()[stopword_features].fillna(method='backfill')
stopword_test = x_test.copy()[stopword_features].fillna(method='backfill')

In [ ]:
model_type = ['linreg', 'randforest']
label_type = ['volume', 'close']
stock_type = ['AAPL', 'GOOG', 'NOK']

model = model_type[0]
label = label_type[0]
stock = 

if model == 'linreg':
    model = LinearRegression()
else:
    model = RandomForestClassifier(max_depth=100, min_samples_leaf=15, n_estimators=100)
    


# Model 3: N-grams

In [38]:
ngram_train = x_train.copy()[ngram_features].fillna(method='backfill')

# Model 4: All Features

---

# DRIVER CODE

In [407]:
# VAR DECLARATIONS
AAPL_volume_train = y_train.copy()['AAPLVolume']
AAPL_close_train = y_train.copy()['AAPLClose']
AAPL_volume_test = y_test.copy()['AAPLVolume']
AAPL_close_test = y_test.copy()['AAPLClose']

GOOG_volume_train = y_train.copy()['GOOGVolume']
GOOG_close_train = y_train.copy()['GOOGClose']
GOOG_volume_test = y_test.copy()['GOOGVolume']
GOOG_close_test = y_test.copy()['GOOGClose']

NOK_volume_train = y_train.copy()['NOKVolume']
NOK_close_train = y_train.copy()['NOKClose']
NOK_volume_test = y_test.copy()['NOKVolume']
NOK_close_test = y_test.copy()['NOKClose']

sentiment_features = list_x_features[2:21]
stopword_features = list_x_features[21:57]
ngram_features = list_x_features[57:]
all_features = list_x_features[2:]


best_features = ['unigram-43', 'Total_Sentiment_Sum_Last_3_Time_Period',
       'Mean_Sentiment_Sum_Last_3_Time_Period',
       'Min_Sentiment_Last_3_Time_Period',
       'Mean_Fraction_Neutral_Last_3_Time_Period', 'bigram-mean',
       'Max_Sentiment_Last_3_Time_Period',
       'Mean_Fraction_Negative_Last_3_Time_Period', 'Sentiment_Max',
       'Fraction_Negative', 'Mean_Sentiment_Last_3_Time_Period',
       'Fraction_Positive', 'Mean_Fraction_of_post_BUY',
       'Mean_Fraction_Positive_Last_3_Time_Period', 'Fraction_Neutral',
       'Mean_Fraction_of_post_SELL', 'Mean_Fraction_of_post_LONG',
       'unigram-24', 'Mean_BUY_Occurences', 'Mean_SELL_Occurences',
       'bigram-max', 'Mean_LONG_Occurences', 'unigram-48', 'unigram-31',
       'Total_Number_Negative', 'unigram-max', 'Mean_SHORT_Occurences',
       'Sentiment_Mean', 'Mean_Fraction_of_post_SHORT', 'Total_BUY_Occurences',
       'unigram-20', 'unigram-44', 'Max_Fraction_of_post_LONG',
       'Sentiment_Sum', 'Max_Fraction_of_post_BUY', 'unigram-23', 'unigram-42',
       'unigram-15', 'unigram-11', 'unigram-33', 'unigram-16', 'unigram-41',
       'unigram-39', 'unigram-30', 'unigram-22', 'unigram-4', 'unigram-18',
       'unigram-mean', 'unigram-38', 'bigram-28']

In [223]:
# DICT DECLARATIONS
driver_dict = {
    'AAPL': {
        'volume': [AAPL_volume_train, AAPL_volume_test],
        'close': [AAPL_close_train, AAPL_close_test]
    },
    'GOOG': {
        'volume': [GOOG_volume_train, GOOG_volume_test],
        'close': [GOOG_close_train, GOOG_close_test]
    },
    'NOK': {
        'volume': [NOK_volume_train, NOK_volume_test],
        'close': [NOK_close_train, NOK_close_test]
    }
}

In [562]:
# -----------------------------------------------------------------
feature_type = ['sentiment', 'stopword', 'ngram', 'all', 'best']
model_type = ['linreg', 'randforest']
label_type = ['volume', 'close']
stock_type = ['AAPL', 'GOOG', 'NOK']

feature = feature_type[4]
model = model_type[1]
label = label_type[1]
stock = stock_type[2]
# -----------------------------------------------------------------

if feature == 'sentiment':
    feature_train = x_train.copy()[sentiment_features].fillna(method='backfill')
    feature_test = x_test.copy()[sentiment_features].fillna(method='backfill')
elif feature == 'stopword':
    feature_train = x_train.copy()[stopword_features].fillna(method='backfill')
    feature_test = x_test.copy()[stopword_features].fillna(method='backfill')
elif feature == 'ngram':
    feature_train = x_train.copy()[ngram_features].fillna(method='backfill')
    feature_test = x_test.copy()[ngram_features].fillna(method='backfill')
elif feature == 'best':
    feature_train = x_train.copy()[best_features].fillna(method='backfill')
    feature_test = x_test.copy()[best_features].fillna(method='backfill')
else:
    feature_train = x_train.copy()[all_features].fillna(method='backfill')
    feature_test = x_test.copy()[all_features].fillna(method='backfill')

    
training_data = driver_dict[stock][label][0]
test_data = driver_dict[stock][label][1]
    

print('running {} analysis for {} {} using {}\n'.format(feature.upper(), stock.upper(), label.upper(), model.upper()))

if model == 'linreg':
    model = LinearRegression()
    lin_model.fit(feature_train, training_data)
    pred = lin_model.predict(feature_test)
    
    rounded_pred = pred.copy()
    rounded_pred[rounded_pred > 0.5] = 1
    rounded_pred[rounded_pred <= 0.5] = 0
    
    # Accuacy score
    print('Accuracy:', accuracy_score(test_data, rounded_pred))
    print('Precision:', precision_score(test_data, rounded_pred))
    print('Recall:', recall_score(test_data, rounded_pred))
    print('F1:', f1_score(test_data, rounded_pred))
    print('AUC:', roc_auc_score(test_data, rounded_pred))
    print()
    print('Confusion Matrix:\n', confusion_matrix(test_data, rounded_pred))
    print('Report:\n', classification_report(test_data, rounded_pred))
    
    
else:
    model = RandomForestClassifier(max_depth=100, min_samples_leaf=15, n_estimators=100)
    model.fit(feature_train, training_data)
    pred = model.predict(feature_test)

    # Accuacy score
    print('Accuracy:', accuracy_score(test_data, pred))
    print('Precision:', precision_score(test_data, pred))
    print('Recall:', recall_score(test_data, pred))
    print('F1:', f1_score(test_data, pred))
    print('AUC:', roc_auc_score(test_data, pred))
    print()
    print('Confusion Matrix:\n', confusion_matrix(test_data, pred))
    print('Report:\n', classification_report(test_data, pred))


running BEST analysis for NOK CLOSE using RANDFOREST

Accuracy: 0.5619047619047619
Precision: 0.573170731707317
Recall: 0.8103448275862069
F1: 0.6714285714285714
AUC: 0.5328319882611886

Confusion Matrix:
 [[12 35]
 [11 47]]
Report:
               precision    recall  f1-score   support

           0       0.52      0.26      0.34        47
           1       0.57      0.81      0.67        58

    accuracy                           0.56       105
   macro avg       0.55      0.53      0.51       105
weighted avg       0.55      0.56      0.52       105

